In [16]:
%%time
import pandas as pd
from sklearn import cross_validation
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.linear_model import Lasso
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.naive_bayes import GaussianNB
#from sklearn.svm import SVC
from sklearn import ensemble
#from sklearn.naive_bayes import BernoulliNB
#from datetime import datetime
#from blaze import Data
#import dask.dataframe as dd
import dask.array as da

CPU times: user 669 µs, sys: 862 µs, total: 1.53 ms
Wall time: 3.85 ms


In [2]:
%%time
train_dtypes = {'date_time': pd.np.object,
'site_name': pd.np.int64,
'posa_continent': pd.np.int64,
'user_location_country': pd.np.int64,
'user_location_region': pd.np.int64,
'user_location_city': pd.np.int64,
'orig_destination_distance': pd.np.float64,
'user_id': pd.np.int64,
'is_mobile': pd.np.int64,
'is_package': pd.np.int64,
'channel': pd.np.int64,
'srch_ci': pd.np.object,
'srch_co': pd.np.object,
'srch_adults_cnt': pd.np.int64,
'srch_children_cnt': pd.np.int64,
'srch_rm_cnt': pd.np.int64,
'srch_destination_id': pd.np.int64,
'srch_destination_type_id': pd.np.int64,
'is_booking': pd.np.int64,
'cnt': pd.np.int64,
'hotel_continent': pd.np.int64,
'hotel_country': pd.np.int64,
'hotel_market': pd.np.int64,
'hotel_cluster': pd.np.int64}

CPU times: user 18 µs, sys: 10 µs, total: 28 µs
Wall time: 26 µs


In [3]:
%%time
all_train = pd.read_csv('train.csv', dtype=train_dtypes)
#temp_train = pd.read_csv('train.csv', dtype=train_dtypes, iterator=True, chunksize=1000)
#all_train = pd.concat(temp_train, ignore_index=True)

CPU times: user 1min 58s, sys: 1min 33s, total: 3min 32s
Wall time: 3min 45s


In [4]:
%%time
#all_train['orig_destination_distance'] = all_train['orig_destination_distance'].fillna(all_train['orig_destination_distance'].median()).astype(int)
#all_train['date_time'] = pd.to_datetime(all_train['date_time'], errors='coerce')
#all_train['srch_ci'] = pd.to_datetime(all_train['srch_ci'], errors='coerce')
#all_train['srch_co'] = pd.to_datetime(all_train['srch_co'], errors='coerce')
#Remove dates columns
all_train = all_train.drop(['date_time','orig_destination_distance','srch_ci','srch_co','is_booking', 'cnt'], 1)

CPU times: user 3.1 s, sys: 12 s, total: 15.1 s
Wall time: 18.6 s


In [5]:
%%time
#Split into training and test data
#split = int(0.70*len(all_train))
#all_train = all_train[split:]
#train = all_train[0:split]
#test  = all_train[split:]
features = all_train.ix[:,:'hotel_market'] 
labels = all_train.ix[:,'hotel_cluster':]
#features_test = test.ix[:,:'hotel_market'] 
#labels_test = test.ix[:,'hotel_cluster':]
#features_train = da.from_array(features_train.values, chunks=1000)
#labels_train = da.from_array(labels_train.values, chunks=1000)
#features_train = features_train.values
#labels_train = labels_train.values
#features_test = features_test.values
#labels_test = labels_test.values

CPU times: user 930 µs, sys: 1.45 ms, total: 2.38 ms
Wall time: 2.3 ms


In [6]:
%%time
features_train, features_test, labels_train, labels_test = cross_validation.train_test_split(features, labels, test_size=0.25, random_state=42)


CPU times: user 46.9 s, sys: 20.2 s, total: 1min 7s
Wall time: 1min 13s


In [11]:
%%time
selector = SelectKBest(k=5)
#.fit_transform(X, y)
#X_new.shape

selector.fit(features_train, labels_train.values.ravel())

CPU times: user 25.6 s, sys: 22.9 s, total: 48.5 s
Wall time: 56.7 s


In [12]:
%%time
feature_list = selector.transform(features_train)

CPU times: user 488 ms, sys: 1.57 s, total: 2.06 s
Wall time: 2.17 s


In [13]:
feature_list

array([[  2,   3,   0,   2,  50],
       [  2,   3,   0,   2,  50],
       [  2,   3,   0,   2,  50],
       ..., 
       [  2,   3,   0,   2,  50],
       [  2,   3,   0,   3,  99],
       [ 29,   1,   0,   6, 144]])

In [14]:
features_train.head()

,site_name,posa_continent,user_location_country,user_location_region,user_location_city,user_id,is_mobile,is_package,channel,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,hotel_continent,hotel_country,hotel_market
36259822,2,3,66,174,27251,875345,0,0,3,2,0,1,9147,1,2,50,367
15546518,2,3,66,459,16458,508143,0,0,3,2,0,1,11854,1,2,50,700
23893709,2,3,66,311,48834,415294,1,0,9,3,1,1,8846,1,2,50,1516
32750507,2,3,66,258,23555,731118,0,0,9,2,2,2,20996,1,6,204,1762
29026052,11,3,205,135,27655,530702,1,0,9,2,0,1,8250,1,2,50,628


In [18]:
%%time
clf = GaussianNB()
#clf = ensemble.AdaBoostClassifier(n_estimators=10)
#clf = ensemble.AdaBoostClassifier(SVC(probability=True, kernel='linear'),n_estimators=10)
#clf = ensemble.GradientBoostingClassifier(SVC(probability=True, kernel='linear'),n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
#clf = ensemble.GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0, verbose=3)
#clf = ensemble.RandomForestClassifier(n_estimators=10)
#clf - BernoulliNB()
clf = clf.fit(feature_list, labels_train.values.ravel())

CPU times: user 18.1 s, sys: 2.43 s, total: 20.5 s
Wall time: 21.3 s


In [19]:
%%time
feature_test_list = selector.transform(features_test)
pred = clf.predict(feature_test_list)

CPU times: user 1min 13s, sys: 2min 52s, total: 4min 6s
Wall time: 9min 55s


In [20]:
print accuracy_score(pred, labels_test.values.ravel())

0.0454018200441


In [ ]:
test_dtypes = {'id': pd.np.int64,
'date_time': pd.np.object,
'site_name': pd.np.int64,
'posa_continent': pd.np.int64,
'user_location_country': pd.np.int64,
'user_location_region': pd.np.int64,
'user_location_city': pd.np.int64,
'orig_destination_distance': pd.np.float64,
'user_id': pd.np.int64,
'is_mobile': pd.np.int64,
'is_package': pd.np.int64,
'channel': pd.np.int64,
'srch_ci': pd.np.object,
'srch_co': pd.np.object,
'srch_adults_cnt': pd.np.int64,
'srch_children_cnt': pd.np.int64,
'srch_rm_cnt': pd.np.int64,
'srch_destination_id': pd.np.int64,
'srch_destination_type_id': pd.np.int64,
'hotel_continent': pd.np.int64,
'hotel_country': pd.np.int64,
'hotel_market': pd.np.int64}

In [ ]:
%%time
all_test = pd.read_csv('test.csv', dtype=test_dtypes)

In [ ]:
all_test['orig_destination_distance'] = all_test['orig_destination_distance'].fillna(0.0).astype(int)
#all_train['date_time'] = pd.to_datetime(all_train['date_time'], errors='coerce')
#all_train['srch_ci'] = pd.to_datetime(all_train['srch_ci'], errors='coerce')
#all_train['srch_co'] = pd.to_datetime(all_train['srch_co'], errors='coerce')
#Remove dates columns
testing_file = all_test.drop(['id', 'date_time','srch_ci','srch_co'], 1)
testing_file = testing_file.values

In [ ]:
%%time
probs = pd.DataFrame(clf.predict_proba(testing_file))

In [ ]:
%%time
probs_series = pd.Series([(i, r.sort_values(ascending=False)[:5].index.values) for i,r in probs.iterrows()])
probs_series = probs_series.values

In [ ]:
%%time
indices = [a for a,b in probs_series]
values = [b for a,b in probs_series]

In [ ]:
%%time
submission = pd.DataFrame()
submission['id'] = indices
submission['hotel_cluster'] = [' '.join(str(x) for x in y) for y in values]

In [ ]:
%%time
submission.to_csv('submission.csv', index=False)